In [2]:
# Calculate Einstein Mean Squared Displacement using MDAnalysis for crystal forming (type 1) and amorphous (type 2) coarse grain silk particles.
# by Jacob Graham
# Sinan Keten's Computational Nanodynamics Laboratory

In [3]:
import MDAnalysis as mda
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl

In [4]:
import MDAnalysis.analysis.msd as msd

In [5]:
# Set location of all simulations to be analyzed.
sim_location = f'{os.getcwd()}'
print(sim_location)

/projects/p31412/DPD_Silk_Order_Study/Correct_Domain_Ratio/DPD-Silk-Analysis-Package/Simulation_Analysis


# Define a Function to Compute Einstein MSD with MDAnalysis for Type 1 and Type 2 Particles Separately

In [6]:
# Function that computes Einstein MSD for type 1 and type 2 atoms for a given MDAnalysis Universe.
# Apparently types 1,2, ... in the psf file of this Universe are not recognized by MDAnalysis.
# Instead, the type intformation is stored as unique atom names.

def get_msd_by_type(u, start, stop):
    """A Function to calculate Einstein Mean Squared Displacements for type 1 and type 2 particles separately.
    Requires:
    import MDAnalysis as mda
    import MDAnalysis.analysis.msd as msd
    
    Returns:
    msd_results_type1 - MSD data for particles of type (and name) 1 over the trajectory stored in universe, u.
    msd_results_type2 - MSD data for particles of type (and name) 2 over the trajectory stored in universe, u.
    """
    def __init__(self, u, start, stop):
        self.u = u
        self.start = start
        self.stop = stop
    
    # Get the mean square displacement of name 1 and name 2 atoms in this Universe over a specified range of frames.
    msd_analysis_type1 = msd.EinsteinMSD(u, select='name 1')
    msd_analysis_type2 = msd.EinsteinMSD(u, select='name 2')
    
    # Run the analyses.
    msd_analysis_type1.run(start=start,stop=stop)
    msd_analysis_type2.run(start=start,stop=stop)
    
    # Access the mean square displacement data
    msd_results_type1 = msd_analysis_type1.results.timeseries
    msd_results_type2 = msd_analysis_type2.results.timeseries
    
    # Return both sets of msd data.
    return msd_results_type1, msd_results_type2

In [ ]:
# Set simulation-specific variables specifying random seed used and dcd files to be analyzed.
randseed = '11111'
psf_protein_only = 'spider_HA1B1A1B1x2134_silkworm_A1B1x0_protein_only.psf'
msd_dcd = f'equil_after_shear_{randseed}_unwrap.dcd'

# Specify some values for unit conversion.
dumpfreq = 1 # Specify number of timesteps between frames passed to the msd calculation.
timestep = 0.03 # LJ units
ns_per_tau = 0.75 # Conversion ratio for tau to ns (0.75 ns/tau)
ns_per_frame = dumpfreq*timestep*ns_per_tau

# Load 'this_sim' into an MDAnalysis Universe.
u = mda.Universe(f'{sim_location}/{this_sim}/{psf_protein_only}', 
            [f'{sim_location}/{this_sim}/{msd_dcd}'],in_memory=True)

# Get msd data by passing universe u into function get_msd_by_type.
print(len(u.trajectory))
nframes = len(u.trajectory)
msd_results_type1, msd_results_type2 = get_msd_by_type(u, start=0, stop=nframes)

# Append new msd data to lists.
msd_type1.append(msd_results_type1)
msd_type2.append(msd_results_type2)

In [ ]:
# Plot the result.
plt.figure(figsize=(16,14))
colors = ['red','green','blue','grey', 'purple']
# Plot the MSD stored in msd_type1 and msd_type2
plt.plot(np.arange(nframes)*ns_per_frame, msd_type1[i]*9.321*9.321,ls="-", c=colors[i], label=f'Type 1 (Crystalline)')
plt.plot(np.arange(nframes)*ns_per_frame, msd_type2[i]*9.321*9.321,ls="--", c=colors[i], label=f'Type 2 (Amorphous)')
plt.ylabel('MSD (A$^2$)')
plt.xlabel('Time (ns)')
plt.legend()
plt.show()